<a href="https://colab.research.google.com/github/lordbvico/msci-641-final-project/blob/main/task1_rerun_final_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
import pandas as pd
id = "1AYls0rBsSfK9HsWWj8kdN6prEJcZsFEk"
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('train.jsonl')
df = pd.read_json('train.jsonl', lines=True)


In [ ]:
id1 = "18yu4xEsU1W_DYlT4dE6fCeXIvFJ_JSWQ"
downloaded1 = drive.CreateFile({'id':id1})
downloaded1.GetContentFile('val.jsonl')
df2 = pd.read_json('val.jsonl', lines=True)

In [ ]:
id2 = "1z3X-ZrhbwIiabENW69nBUmj8O0Mb-_Er"
downloaded2 = drive.CreateFile({'id':id2})
downloaded2.GetContentFile('test.jsonl')
df3 = pd.read_json('test.jsonl', lines=True)

In [ ]:
!pip install wandb -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.6/215.6 kB 23.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.3 MB/s eta 0:00:00


In [ ]:
# Log in to your W&B account
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from transformers import RobertaTokenizer, RobertaModel
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
import wandb
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer

In [ ]:
sweep_config = {
        "name": "sweep",
        "method": "random",
        "metric": {"name": "Validation F1 Score", "goal": "maximize"},
        "parameters": {
            'learning_rate': {'max': 0.00001, 'min': 0.000001},
            "num_epochs": {"values": [3,4,5,7]},
            "hidden_size": {"value": 768},
            #'learning_rate': {'value':0.00000892589530014827},
            #"num_epochs": {"value": 3},
            #"hidden_size": {"value": 768},
            #'weight_decay': {'value':0.0031730901067438273 }
            #"weight_decay": {"values": [0.3,0.4,0.5]}
            #'weight_decay': {'max': 0.5, 'min': 0.2}
            'weight_decay': {'max': 0.05, 'min': 0.001}

        }
    }

    # Initialize Wandb sweep
sweep_id = wandb.sweep(sweep=sweep_config, project="Roberta Param Sweep")

Create sweep with ID: yhnisnkq
Sweep URL: https://wandb.ai/team_vico/Roberta%20Param%20Sweep%20Rerun%20Testing/sweeps/yhnisnkq


In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(w) for w in text]


def preprocess_data(df):
    df = df[['postText', 'targetTitle']]

    # Convert all columns into strings
    df.loc[:, ['postText', 'targetTitle']] = df.loc[:, ['postText', 'targetTitle']].astype(str)

    # Tokenize the relevant columns (not actually used for the Bag of Word approach)
    tokenizer = RegexpTokenizer(r"\w+")
    df["postText_tokens"] = df.apply(lambda row: tokenizer.tokenize(row["postText"]), axis=1)
    df["paragraph_tokens"] = df.apply(lambda row: tokenizer.tokenize(row["targetTitle"]), axis=1)
    #df["targetTitle_tokens"] = df.apply(lambda row: tokenizer.tokenize(row["targetTitle"]), axis=1)

       # Lowercasing
    df['postText_tokens'] = df['postText_tokens'].map(lambda row: list(map(str.lower, row)))
    df['paragraph_tokens'] = df['paragraph_tokens'].map(lambda row: list(map(str.lower, row)))
    #df['targetTitle_tokens'] = df['targetTitle_tokens'].map(lambda row: list(map(str.lower, row)))

    # Multiple space to single space and remove special characters
    #df[['postText_tokens', 'paragraph_tokens', 'targetTitle_tokens']] = df[['postText_tokens', 'paragraph_tokens', 'targetTitle_tokens']].replace(r'\s+', ' ', regex=True).replace(r'\W', ' ', regex=True)
    df[['postText_tokens', 'paragraph_tokens']] = df[['postText_tokens', 'paragraph_tokens']].replace(r'\s+', ' ', regex=True).replace(r'\W', ' ', regex=True)

    # Lemmatize tokens
    df['postText_tokens'] = df['postText_tokens'].apply(lemmatize_text)
    df['paragraph_tokens'] = df['paragraph_tokens'].apply(lemmatize_text)
    #df['targetTitle_tokens'] = df['targetTitle_tokens'].apply(lemmatize_text)

    #df['combined_texts'] = df['postText_tokens'].apply(lambda tokens: ' '.join(tokens)) + " " + df['paragraph_tokens'].apply(lambda tokens: ' '.join(tokens)) + " " + df['targetTitle_tokens'].apply(lambda tokens: ' '.join(tokens))
    df['combined_texts'] = df['postText_tokens'].apply(lambda tokens: ' '.join(tokens)) + " " + df['paragraph_tokens'].apply(lambda tokens: ' '.join(tokens))
    #df['combined_texts'] = df['postText_tokens'].apply(lambda tokens: ' '.join(tokens))

    return df

In [ ]:
def encode_labels(train_labels):
    flattened_labels = [label for sublist in train_labels for label in sublist]
    label_encoder = LabelEncoder()
    label_encoder.fit(flattened_labels)
    encoded_labels = label_encoder.transform(flattened_labels)
    return encoded_labels, label_encoder

In [ ]:
def encode_texts(texts, tokenizer, max_seq_length):
    input_ids = []
    attention_masks = []

    for text in texts:
        encoding = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_seq_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        input_ids.append(encoding['input_ids'].squeeze())
        attention_masks.append(encoding['attention_mask'].squeeze())

    input_ids = torch.stack(input_ids)
    attention_masks = torch.stack(attention_masks)

    return input_ids, attention_masks

In [ ]:
# lastest
class RoBERTaClassifier(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(RoBERTaClassifier, self).__init__()
        self.roberta = RobertaModel.from_pretrained('roberta-base')
        self.dropout = nn.Dropout(0.1)
        self.fc1 = nn.Linear(hidden_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, input_ids, attention_mask):
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        #pooled_output = self.fc1(pooled_output)
        pooled_output = F.relu(self.fc1(pooled_output)) # Added activation function
        logits = self.fc2(pooled_output)

        #return logits
        return F.softmax(logits, dim=1)

In [ ]:
train_selected = preprocess_data(df)
val_selected = preprocess_data(df2)
test_selected = preprocess_data(df3)

X_train = train_selected['combined_texts']
X_val = val_selected['combined_texts']
X_test = test_selected['combined_texts']

<ipython-input-12-90efcd88b1a8>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, ['postText', 'targetTitle']] = df.loc[:, ['postText', 'targetTitle']].astype(str)
<ipython-input-12-90efcd88b1a8>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["postText_tokens"] = df.apply(lambda row: tokenizer.tokenize(row["postText"]), axis=1)
<ipython-input-12-90efcd88b1a8>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

In [ ]:
train_labels, label_encoder = encode_labels(df['tags'].tolist())
val_labels, vlabel_encoder = encode_labels(df2['tags'].tolist())

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
input_size = tokenizer.vocab_size
max_seq_length = 60

In [ ]:
train_input_ids, train_attention_masks = encode_texts(X_train, tokenizer, max_seq_length)
val_input_ids, val_attention_masks = encode_texts(X_val, tokenizer, max_seq_length)
test_input_ids, test_attention_masks = encode_texts(X_test, tokenizer, max_seq_length)
train_labels = torch.tensor(train_labels, dtype=torch.long)
val_labels = torch.tensor(val_labels, dtype=torch.long)

train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataset = TensorDataset(val_input_ids, val_attention_masks, val_labels)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)


In [ ]:
def main():
  run = wandb.init()
  #best_val_accuracy = 0.0
  best_val_f1 = 0.0
  best_model_state = None
  #hidden_size = 768
  output_size = len(label_encoder.classes_)
  #learning_rate = 1e-5 0.05207827445417903

  hidden_size = wandb.config.hidden_size
  learning_rate = wandb.config.learning_rate
  weight_decay = wandb.config.weight_decay


  model = RoBERTaClassifier(hidden_size, output_size)
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model = model.to(device)

  # Define the focal loss function
  #gamma = 2  # The focal loss gamma parameter
  #alpha = None  # The focal loss alpha parameter (optional)
  #criterion = FocalLoss()

  criterion = nn.CrossEntropyLoss()
  optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
  #optimizer = optim.RMSprop(model.parameters(), lr=learning_rate)



  #num_epochs = 5
  num_epochs = wandb.config.num_epochs

  for epoch in range(num_epochs):
      model.train()
      for input_ids, attention_masks, labels in tqdm(train_loader):
          input_ids = input_ids.to(device)
          attention_masks = attention_masks.to(device)
          labels = labels.to(device)

          optimizer.zero_grad()
          logits = model(input_ids, attention_masks)
          loss = criterion(logits, labels)
          loss.backward()
          optimizer.step()

      model.eval()
      val_loss = 0.0
      val_correct = 0
      total_samples = 0
     # epsilon = 0.0000001
      val_outputs = []

      with torch.no_grad():
          all_labels = []
          all_predictions = []
         # true_positive = 0
         # false_positive = 0
         # false_negative = 0
         # true_negative = 0


          for input_ids, attention_masks, v_labels in tqdm(val_loader):
              input_ids = input_ids.to(device)
              attention_masks = attention_masks.to(device)
              v_labels = v_labels.to(device)

              logits = model(input_ids, attention_masks)
              loss = criterion(logits, v_labels)
              val_loss += loss.item()
              _, predicted = torch.max(logits, 1)
              val_correct += (predicted == v_labels).sum().item()
              total_samples +=v_labels.size(0)
              val_outputs.append(predicted)
      val_outputs = torch.cat(val_outputs, dim=0)
      val_predicted = val_outputs.cpu()
      val_converted_array = val_predicted.numpy()
      val_converted_array_tags = label_encoder.inverse_transform(val_converted_array)
      v_output_data = []

      for index, pred in enumerate(val_converted_array_tags):
          prediction1 = {'id': index, 'spoilerType': pred}
          v_output_data.append(prediction1)

      v_out_df = pd.DataFrame(v_output_data)
      #out_df.to_csv("output_big_bird_jul21.csv", index=False)
      filename = f"output_roberta_val_aug4c.csv"
      v_out_df.to_csv(filename, index=False)

              # Update TP, FP, FN, and TN based on predictions and true labels
             # true_positive += ((predicted == 1) & (v_labels == 1)).sum().item()
             # false_positive += ((predicted == 1) & (v_labels == 0)).sum().item()
             # false_negative += ((predicted == 0) & (v_labels == 1)).sum().item()
             # true_negative += ((predicted == 0) & (v_labels == 0)).sum().item()
              #val_correct += (predicted == v_labels).sum().item()
         # try:
        #      precision = true_positive / (true_positive + false_positive)
       #   except ZeroDivisionError:
       #       precision = 0.0

       #   try:
        #      recall = true_positive / (true_positive + false_negative)
       #   except ZeroDivisionError:
       #       recall = 0.0

       #   try:
       #       f1 = 2 * precision * recall / (precision + recall)
       #   except ZeroDivisionError:
       #       f1 = 0.0

       #   accuracy = (true_positive + true_negative) / (true_positive + false_positive + false_negative + true_negative)

        #  print(f'Validation Accuracy: {accuracy}')
       #   print(f'Validation Precision: {precision}')
       #   print(f'Validation Recall: {recall}')
       #   print(f'Validation F1 Score: {f1}')



      all_labels.extend(v_labels.detach().cpu().numpy())
      all_predictions.extend(predicted.detach().cpu().numpy())

      val_loss /= total_samples
      val_accuracy = val_correct / total_samples
      accuracy = accuracy_score(all_labels, all_predictions)
      precision = precision_score(all_labels, all_predictions, average='weighted', zero_division=0)
      recall = recall_score(all_labels, all_predictions, average='weighted', zero_division=0)
      f1 = f1_score(all_labels, all_predictions, average='weighted', zero_division=0)

      if f1 > best_val_f1:
          best_val_f1 = f1
          best_model_state = model.state_dict()




      print(f'Validation Accuracy: {accuracy}')
      print(f'Validation Precision: {precision}')
      print(f'Validation Recall: {recall}')
      print(f'Validation F1 Score: {f1}')

      print(f"Epoch [{epoch+1}/{num_epochs}], Validation Accuracy: {val_accuracy:.4f}")
      print()
      #wandb.log({"Validation Accuracy": accuracy,"Validation Precision": precision,"Validation Recall": recall,"Validation F1 Score": f1,"Val Accuracy": val_accuracy, "Weight Decay": weight_decay, "Validation Loss": val_loss, "Learning Rate": learning_rate, "Number of Epochs": num_epochs, "Hidden Size": hidden_size})
      wandb.log({"Validation Accuracy": accuracy,"Validation Precision": precision,"Validation Recall": recall,"Validation F1 Score": f1,"Val Accuracy": val_accuracy, "Validation Loss": val_loss, "Learning Rate": learning_rate, "Number of Epochs": num_epochs, "Hidden Size": hidden_size})

      print(best_val_f1)

  best_model_filename = f"best_model_roberta_aug4c_lr_{learning_rate}_epochs_{num_epochs}.pth"
  torch.save(best_model_state, best_model_filename)
  model.load_state_dict(torch.load(best_model_filename))
  model.eval()

  test_batch_size = 16
  test_dataset = TensorDataset(test_input_ids, test_attention_masks)
  test_loader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False)
  test_outputs = []

  with torch.no_grad():
    for input_ids, attention_masks in tqdm(test_loader):
      input_ids = input_ids.to(device)
      attention_masks = attention_masks.to(device)

      batch_outputs = model(input_ids, attention_masks)
      _, predicted = torch.max(batch_outputs, 1)
      test_outputs.append(predicted)
  test_outputs = torch.cat(test_outputs, dim=0)
  test_predicted = test_outputs.cpu()
  converted_array = test_predicted.numpy()
  converted_array_tags = label_encoder.inverse_transform(converted_array)
  output_data = []

  for index, pred in enumerate(converted_array_tags):
      prediction = {'id': index, 'spoilerType': pred}
      output_data.append(prediction)

  out_df = pd.DataFrame(output_data)
  #out_df.to_csv("output_big_bird_jul21.csv", index=False)
  filename = f"output_roberta_lr_{learning_rate}_epochs_{num_epochs}_aug4c.csv"
  out_df.to_csv(filename, index=False)

if __name__ == "__main__":
  wandb.agent(sweep_id, function=main)




wandb: Agent Starting Run: tj6wrxqm with config:
wandb: 	hidden_size: 768
wandb: 	learning_rate: 8.92589530014827e-06
wandb: 	num_epochs: 3
wandb: 	weight_decay: 0.0031730901067438273
wandb: Currently logged in as: lordbvico (team_vico). Use `wandb login --relogin` to force relogin


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 25/25 [00:01<00:00, 18.20it/s]


Validation Accuracy: 0.4375
Validation Precision: 0.38541666666666663
Validation Recall: 0.4375
Validation F1 Score: 0.3720588235294118
Epoch [1/3], Validation Accuracy: 0.5200

0.3720588235294118


100%|██████████| 25/25 [00:01<00:00, 17.33it/s]


Validation Accuracy: 0.5625
Validation Precision: 0.7375
Validation Recall: 0.5625
Validation F1 Score: 0.5435606060606061
Epoch [2/3], Validation Accuracy: 0.5850

0.5435606060606061


...

Hidden Size,▁▁▁
Learning Rate,▁▁▁
Number of Epochs,▁▁▁
Val Accuracy,▁▇█
Validation Accuracy,▁▆█
Validation F1 Score,▁▆█
Validation Loss,█▂▁
Validation Precision,▁▇█
Validation Recall,▁▆█
Hidden Size,768
Learning Rate,1e-05


wandb: Agent Starting Run: 7p78lbfz with config:
wandb: 	hidden_size: 768
wandb: 	learning_rate: 8.92589530014827e-06
wandb: 	num_epochs: 3
wandb: 	weight_decay: 0.0031730901067438273


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 25/25 [00:01<00:00, 16.76it/s]


Validation Accuracy: 0.5
Validation Precision: 0.375
Validation Recall: 0.5
Validation F1 Score: 0.41666666666666663
Epoch [1/3], Validation Accuracy: 0.5675

0.41666666666666663


 48%|████▊     | 95/200 [00:17<00:19,  5.33it/s]wandb: Ctrl + C detected. Stopping sweep.
